# Munging the inaugural addresses


In [1]:
import urllib.request
# html = urllib.request.urlopen('http://avalon.law.yale.edu/subject_menus/inaug.asp', data = None)

In [2]:
html = urllib.request.urlopen('http://avalon.law.yale.edu/subject_menus/inaug.asp', data = None)

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')


In [4]:
soup.title

<title>Avalon Project - The Inaugural Addresses of the Presidents</title>

In [5]:
links = []
dates = []
pres = []
url_prefix = 'http://avalon.law.yale.edu/'
import re
for link in soup.find_all('a'):
    raw_link = link.get('href')
#     print(raw_link)
    if re.search('subject_menus', raw_link):
        continue
    elif re.search('about/purpose', raw_link):
        continue
    elif re.search('default', raw_link):
        continue
    elif re.match('http', raw_link):
        continue            
    else:
        links.append(url_prefix + raw_link[3:])
        dates.append(link.text)
        pres.append(link.find_previous('span').get_text())
    
print(links)
print(dates)
print(pres)

['http://avalon.law.yale.edu/18th_century/wash1.asp', 'http://avalon.law.yale.edu/18th_century/wash2.asp', 'http://avalon.law.yale.edu/18th_century/adams.asp', 'http://avalon.law.yale.edu/19th_century/jefinau1.asp', 'http://avalon.law.yale.edu/19th_century/jefinau2.asp', 'http://avalon.law.yale.edu/19th_century/madison1.asp', 'http://avalon.law.yale.edu/19th_century/madison2.asp', 'http://avalon.law.yale.edu/19th_century/monroe1.asp', 'http://avalon.law.yale.edu/19th_century/monroe2.asp', 'http://avalon.law.yale.edu/19th_century/qadams.asp', 'http://avalon.law.yale.edu/19th_century/jackson1.asp', 'http://avalon.law.yale.edu/19th_century/jackson2.asp', 'http://avalon.law.yale.edu/19th_century/vanburen.asp', 'http://avalon.law.yale.edu/19th_century/harrison.asp', 'http://avalon.law.yale.edu/19th_century/polk.asp', 'http://avalon.law.yale.edu/19th_century/taylor.asp', 'http://avalon.law.yale.edu/19th_century/pierce.asp', 'http://avalon.law.yale.edu/19th_century/buchanan.asp', 'http://aval

In [6]:
import pandas as pd

In [7]:
urls = pd.Series(links)
years = pd.Series(dates)
pres = pd.Series(pres)

inaugurals = pd.concat([urls, years, pres], axis = 1)
inaugurals.columns = ['urls', 'years', 'pres']
inaugurals.head()

urls years               pres
0  http://avalon.law.yale.edu/18th_century/wash1.asp  1789  George Washington
1  http://avalon.law.yale.edu/18th_century/wash2.asp  1793  George Washington
2  http://avalon.law.yale.edu/18th_century/adams.asp  1797         John Adams
3  http://avalon.law.yale.edu/19th_century/jefina...  1801   Thomas Jefferson
4  http://avalon.law.yale.edu/19th_century/jefina...  1805   Thomas Jefferson

In [8]:
# Another Way to get pres, but fails to account for multi-term presidents: 
pres_list = []
for span in soup.find_all('span'):
#     pres = span.get_text()
    pres = span.contents
#     pres_list.append(pres)
    pres_list = pres_list + pres
print(pres_list)

['George Washington', 'John Adams', 'Thomas Jefferson', 'James Madison', 'James Monroe', 'John Quincy Adams', 'Andrew Jackson', 'Martin Van Buren', 'William Henry Harrison', 'James Polk', 'Zachary Taylor', 'Franklin Pierce', 'James Buchanan', 'Abraham Lincoln', 'Ulysses S. Grant', 'Rutherford B. Hayes', 'James A. Garfield', 'Grover Cleveland', 'Benjamin Harrison', 'William McKinley', 'Theodore Roosevelt', 'William Howard Taft', 'Woodrow Wilson', 'Warren G. Harding', 'Calvin Coolidge', 'Herbert Hoover', 'Franklin D. Roosevelt', 'Harry S. Truman', 'Dwight D. Eisenhower', 'John F. Kennedy', 'Lyndon Baines Johnson', 'Richard Milhous Nixon', 'Jimmy Carter', 'Ronald Reagan', 'George Bush', 'William Clinton', 'George W. Bush', 'Barack Obama']


## Now get the texts
Here's where we go in and scrape the texts and add them to a column in our dataframe.

In [9]:
# A function to count the words
def wordcounter(text):
    if "This one did not work:" in text:
        count = "N/A"
    else:
        words = text.split()
        count = 0
        for word in words:
            count += 1
    return count
        

In [10]:
# A function to strip the header from the scraped texts

def remove_str_head(text):
    try:
        start_pos = text.index("21st Century 2000 - ") + len("21st Century 2000 - ")
        try:
            end_pos = text.index("Inaugural Speeches Page")
        except:
            # I found this alternative-- this is why scraping is such a pain!
            end_pos = text.index("Inauguration Speeches Page")
        print('SUCCESS')
        return(text[start_pos:end_pos])
    except:
        print('HEAD NOT STRIPPED')
        not_stripped_count += 1
        return text
        return not_stripped_count
    

In [11]:
# I don't use this function because the one above does a better job:
def remove_head(text):
    text_words_list = text.split()
    start_pos = text_words_list.index("2000")
    # start at '2000', which is the end of the heading on most pages
    print(start_pos)
    sequence = ["Inaugural", "Speeches", "Page"]
    for i in range(len(text_words_list)):
        if text_words_list[i:i+len(sequence)] == sequence:
            end_pos = i
    print(end_pos)
    text_words_list = text_words_list[start_pos:end_pos]
    text.join(text_words_list) # this is wrong
    return text
    print(text)




In [12]:
# A function to strip the texts
def format_text(url):
    html = urllib.request.urlopen(url, data = None)
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text()
    text = remove_str_head(text)
    text = text.strip()
    text = text.replace('\n', ' ').replace('\r', ' ')
    print(text[:50])
    return text

In [13]:
import time
# Don't want to wear out our welcome on the yale server!
texts = []
fail_count = 0
not_stripped_count = 0
for url in inaugurals['urls']:
    print(url)
    try:
        scrape = format_text(url)
        # scrape = remove_str_head(scrape)
        # commented out because I put this in the previous step
        texts.append(scrape)
    except:
        texts.append("This one did not work:" + url)
        fail_count += 1
    time.sleep(1)

print("{} texts failed to load.".format(fail_count))
print("{} heads were not stripped off".format(not_stripped_count))


http://avalon.law.yale.edu/18th_century/wash1.asp
SUCCESS
First Inaugural Address of George Washington  THE 
http://avalon.law.yale.edu/18th_century/wash2.asp
SUCCESS
Second Inaugural Address of George Washington  THE
http://avalon.law.yale.edu/18th_century/adams.asp
SUCCESS
© 2008 Lillian Goldman Law Library 127 Wall Street
http://avalon.law.yale.edu/19th_century/jefinau1.asp
SUCCESS
The Avalon Project  at Yale Law School  Thomas Jef
http://avalon.law.yale.edu/19th_century/jefinau2.asp
SUCCESS
Thomas Jefferson Second Inaugural Address  Second 
http://avalon.law.yale.edu/19th_century/madison1.asp
SUCCESS
The Avalon Project  at Yale Law School James Madis
http://avalon.law.yale.edu/19th_century/madison2.asp
SUCCESS
The Avalon Project  at Yale Law School James Madis
http://avalon.law.yale.edu/19th_century/monroe1.asp
SUCCESS
First Inaugural Address of James Monroe  TUESDAY, 
http://avalon.law.yale.edu/19th_century/monroe2.asp
SUCCESS
Second Inaugural Address of James Monroe   MONDAY,
htt

In [14]:
# Remove the superfluous text before and after the addresses 
# I don't use this function because I built it into the scrape process above.

clean_texts = []
for text in texts:
    try:
        text = text.strip()
        clean_texts.append(text)
    except:
        clean_texts.append("N/A")

print(clean_texts[1])

Second Inaugural Address of George Washington  THE CITY OF PHILADELPHIA MONDAY, MARCH 4, 1793 Fellow Citizens:  I am again called upon by the voice of my country to execute the functions of its Chief Magistrate. When the occasion proper for it shall arrive, I shall endeavor to express the high sense I entertain of this distinguished honor, and of the confidence which has been reposed in me by the people of united America.  Previous to the execution of any official act of the President the Constitution requires an oath of office. This oath I am now about to take, and in your presence: That if it shall be found during my administration of the Government I have in any instance violated willingly or knowingly the injunctions thereof, I may (besides incurring constitutional punishment) be subject to the upbraidings of all who are now witnesses of the present solemn ceremony.


In [22]:
# split and count the words in the texts
# I don't use this function either.
texts = clean_texts

In [15]:
counts = []
for text in texts:
    counts.append(wordcounter(text))
    
print(counts)

[1443, 149, 2417, 1742, 2162, 1193, 1230, 3373, 4470, 2919, 1134, 1181, 'N/A', 8434, 4805, 1096, 3335, 'N/A', 3640, 711, 1132, 1346, 2483, 'N/A', 1689, 2024, 4401, 3967, 2226, 992, 5438, 1709, 1536, 3336, 'N/A', 3758, 1887, 1811, 1329, 557, 2276, 2452, 1647, 1350, 1490, 2115, 1797, 1232, 2438, 2564, 2319, 1601, 2176, 1595, 2073, 2417]


In [16]:
# now add the new list to the dataframe

inaugurals['texts'] = texts
inaugurals['wordcounts'] = counts

In [19]:
inaugurals



urls years  \
0   http://avalon.law.yale.edu/18th_century/wash1.asp  1789   
1   http://avalon.law.yale.edu/18th_century/wash2.asp  1793   
2   http://avalon.law.yale.edu/18th_century/adams.asp  1797   
3   http://avalon.law.yale.edu/19th_century/jefina...  1801   
4   http://avalon.law.yale.edu/19th_century/jefina...  1805   
5   http://avalon.law.yale.edu/19th_century/madiso...  1809   
6   http://avalon.law.yale.edu/19th_century/madiso...  1813   
7   http://avalon.law.yale.edu/19th_century/monroe...  1817   
8   http://avalon.law.yale.edu/19th_century/monroe...  1821   
9   http://avalon.law.yale.edu/19th_century/qadams...  1825   
10  http://avalon.law.yale.edu/19th_century/jackso...  1829   
11  http://avalon.law.yale.edu/19th_century/jackso...  1833   
12  http://avalon.law.yale.edu/19th_century/vanbur...  1837   
13  http://avalon.law.yale.edu/19th_century/harris...  1841   
14   http://avalon.law.yale.edu/19th_century/polk.asp  1845   
15  http://avalon.law.yale.edu/19th_century/taylor...  1849   
16  http://avalon.law.yale.edu/19th_century/pierce...  1853   
17  http://avalon.law.yale.edu/19th_century/buchan...  1857   
18  http://avalon.law.yale.edu/19th_century/lincol...  1861   
19  http://avalon.law.yale.edu/19th_century/lincol...  1865   
20  http://avalon.law.yale.edu/19th_century/grant1...  1869   
21  http://avalon.law.yale.edu/19th_century/grant2...  1873   
22  http://avalon.law.yale.edu/19th_century/hayes.asp  1877   
23  http://avalon.law.yale.edu/19th_century/garfie...  1881   
24  http://avalon.law.yale.edu/19th_century/cleve1...  1885   
25  http://avalon.law.yale.edu/19th_century/cleve2...  1893   
26  http://avalon.law.yale.edu/19th_century/harris...  1889   
27  http://avalon.law.yale.edu/19th_century/mckin1...  1897   
28  http://avalon.law.yale.edu/19th_century/mckin2...  1901   
29  http://avalon.law.yale.edu/20th_century/troos.asp  1905   
30   http://avalon.law.yale.edu/20th_century/taft.asp  1909   
31  http://avalon.law.yale.edu/20th_century/wilson...  1913   
32  http://avalon.law.yale.edu/20th_century/wilson...  1917   
33  http://avalon.law.yale.edu/20th_century/hardin...  1921   
34  http://avalon.law.yale.edu/20th_century/coolid...  1925   
35  http://avalon.law.yale.edu/20th_century/hoover...  1929   
36  http://avalon.law.yale.edu/20th_century/froos1...  1933   
37  http://avalon.law.yale.edu/20th_century/froos2...  1937   
38  http://avalon.law.yale.edu/20th_century/froos3...  1941   
39  http://avalon.law.yale.edu/20th_century/froos4...  1945   
40  http://avalon.law.yale.edu/20th_century/truman...  1949   
41  http://avalon.law.yale.edu/20th_century/eisen1...  1953   
42  http://avalon.law.yale.edu/20th_century/eisen2...  1957   
43  http://avalon.law.yale.edu/20th_century/kenned...  1961   
44  http://avalon.law.yale.edu/20th_century/johnso...  1965   
45  http://avalon.law.yale.edu/20th_century/nixon1...  1969   
46  http://avalon.law.yale.edu/20th_century/nixon2...  1973   
47  http://avalon.law.yale.edu/20th_century/carter...  1977   
48  http://avalon.law.yale.edu/20th_century/reagan...  1981   
49  http://avalon.law.yale.edu/20th_century/reagan...  1985   
50   http://avalon.law.yale.edu/20th_century/bush.asp  1989   
51  http://avalon.law.yale.edu/20th_century/clinto...  1993   
52  http://avalon.law.yale.edu/20th_century/clinto...  1997   
53  http://avalon.law.yale.edu/21st_century/gbush1...  2001   
54  http://avalon.law.yale.edu/21st_century/gbush2...  2005   
55  http://avalon.law.yale.edu/21st_century/obama.asp  2009   

                      pres                                              texts  \
0        George Washington  First Inaugural Address of George Washington  ...   
1        George Washington  Second Inaugural Address of George Washington ...   
2               John Adams  © 2008 Lillian Goldman Law Library 127 Wall St...   
3         Thomas Jefferson  The Avalon Project  at Yale Law School  Thomas...   
4         Thomas Jefferson  Thomas Jefferson Second 

In [17]:
for text in inaugurals['texts']:
    text.strip('\n')
    print(text[:100])
    print(type(text))

First Inaugural Address of George Washington  THE CITY OF NEW YORK THURSDAY, APRIL 30, 1789  Fellow-
<class 'str'>
Second Inaugural Address of George Washington  THE CITY OF PHILADELPHIA MONDAY, MARCH 4, 1793 Fellow
<class 'str'>
© 2008 Lillian Goldman Law Library 127 Wall Street, New Haven, CT 06511.  Avalon Statement of Purpos
<class 'str'>
The Avalon Project  at Yale Law School  Thomas Jefferson First Inaugural Address  First Inaugural Ad
<class 'str'>
Thomas Jefferson Second Inaugural Address  Second Inaugural Address  March 4, 1805  Proceeding, fell
<class 'str'>
The Avalon Project  at Yale Law School James Madison’s First Inaugural Address First Inaugural Addre
<class 'str'>
The Avalon Project  at Yale Law School James Madison’s First Inaugural Address First Inaugural Addre
<class 'str'>
First Inaugural Address of James Monroe  TUESDAY, MARCH 4, 1817 I should be destitute of feeling if 
<class 'str'>
Second Inaugural Address of James Monroe   MONDAY, MARCH 5, 1821 Fellow-Citizens

In [68]:
# Save the dataframe to csv
inaugurals.to_csv('data/inaugurals_may4.csv', index = False)

## Hand edits
I am trying my best to do all of the munging programatically, but a few things are preventing me.

- First, the avalon website is missing a few inaugural addresses, which I have to go fetch by hand.
- Secondly, although I tried to write a program to acquire the president's political parties from tables I found online, I could not figure a way to create a "parties" list to pair with the inaugural addresses. Some presidents of course served two terms, but most lists and tables I tried to scrape don't give multiple instances of the political parties to reflect multiple terms. In other words, I need FDR's "Democrat" to appear 4x in my list, so that when I zip the list together with my existing dataframe, each inaugural address for FDR gets one "Democrat" in the parties column. To make it even more complex, some presidents served without ever giving an inaugural address, etc., rendering one of the best online tables useless for my purposes. I think I could have figured this out eventually, but what's the point-- much easier to hand code.
- Finally, I wanted to add another column, which is New_Party or Incumbent_Party. I am sure I could make such a column in pandas, but it's easier to do this in excel because I know just what LOOKUP formula to use.

So in any case, my next step was to hand-code, and now I have to re-import the edited csv file back into a dataframe to continue my munging and then to move to analysis

In [18]:
import pandas as pd
import os
updated_df = pd.read_csv('./data/inaugurals_may4_incumb.csv')
updated_df.head()

urls  years  \
0  http://avalon.law.yale.edu/18th_century/wash1.asp   1789   
1  http://avalon.law.yale.edu/18th_century/wash2.asp   1793   
2  http://avalon.law.yale.edu/18th_century/adams.asp   1797   
3  http://avalon.law.yale.edu/19th_century/jefina...   1801   
4  http://avalon.law.yale.edu/19th_century/jefina...   1805   

                pres                                              texts  \
0  George Washington  First Inaugural Address of George Washington  ...   
1  George Washington  Second Inaugural Address of George Washington ...   
2         John Adams  © 2008 Lillian Goldman Law Library 127 Wall St...   
3   Thomas Jefferson  The Avalon Project  at Yale Law School  Thomas...   
4   Thomas Jefferson  Thomas Jefferson Second Inaugural Address  Sec...   

   wordcounts                  party           change  
0        1443             Federalist        New_Party  
1         149             Federalist  Incumbent_Party  
2        2417  Democratic-Republican        New_Party  
3        1742  Democratic-Republican  Incumbent_Party  
4        2162  Democratic-Republican  Incumbent_Party

## Add Obama and Trump:
Now that I have the updated df re-imported, I add Obama and Trump with the following code (this would have been easier to hand code, but oh well):

In [20]:
barack3 = [['NaN', 2013, "Barack Obama", "Thank you. Thank you so much.  Vice President Biden, Mr. Chief Justice, Members of the United States Congress, distinguished guests, and fellow citizens:  Each time we gather to inaugurate a President we bear witness to the enduring strength of our Constitution. We affirm the promise of our democracy. We recall that what binds this Nation together is not the colors of our skin or the tenets of our faith or the origins of our names. What makes us exceptional—what makes us American—is our allegiance to an idea articulated in a declaration made more than two centuries ago:  We hold these truths to be self-evident, that all men are created equal; that they are endowed by their Creator with certain unalienable rights; that among these are life, liberty, and the pursuit of happiness.  Today we continue a never-ending journey to bridge the meaning of those words with the realities of our time. For history tells us that while these truths may be self-evident, they've never been self-executing; that while freedom is a gift from God, it must be secured by His people here on Earth. The patriots of 1776 did not fight to replace the tyranny of a king with the privileges of a few or the rule of a mob. They gave to us a republic, a government of and by and for the people, entrusting each generation to keep safe our founding creed.  And for more than 200 years, we have.  Through blood drawn by lash and blood drawn by sword, we learned that no union founded on the principles of liberty and equality could survive half-slave and half-free. We made ourselves anew, and vowed to move forward together.  Together, we determined that a modern economy requires railroads and highways to speed travel and commerce, schools and colleges to train our workers.  Together, we discovered that a free market only thrives when there are rules to ensure competition and fair play.  Together, we resolved that a great nation must care for the vulnerable and protect its people from life's worst hazards and misfortune.  Through it all, we have never relinquished our skepticism of central authority nor have we succumbed to the fiction that all society's ills can be cured through government alone. Our celebration of initiative and enterprise, our insistence on hard work and personal responsibility, these are constants in our character.  But we have always understood that when times change, so must we; that fidelity to our founding principles requires new responses to new challenges; that preserving our individual freedoms ultimately requires collective action. For the American people can no more meet the demands of today's world by acting alone than American soldiers could have met the forces of fascism or communism with muskets and militias. No single person can train all the math and science teachers we'll need to equip our children for the future, or build the roads and networks and research labs that will bring new jobs and businesses to our shores. Now more than ever, we must do these things together, as one nation and one people.  This generation of Americans has been tested by crises that steeled our resolve and proved our resilience. A decade of war is now ending. An economic recovery has begun. America's possibilities are limitless, for we possess all the qualities that this world without boundaries demands: youth and drive; diversity and openness; an endless capacity for risk and a gift for reinvention. My fellow Americans, we are made for this moment and we will seize it—so long as we seize it together.  For we, the people, understand that our country cannot succeed when a shrinking few do very well and a growing many barely make it. We believe that America's prosperity must rest upon the broad shoulders of a rising middle class. We know that America thrives when every person can find independence and pride in their work; when the wages of honest labor liberate families from the brink of hardship. We are true to our creed when a little girl born into the bleakest poverty knows that she has the same chance to succeed as anybody else, because she is an American; she is free and she is equal, not just in the eyes of God, but also in our own.  We understand that outworn programs are inadequate to the needs of our time. So we must harness new ideas and technology to remake our government, revamp our Tax Code, reform our schools, and empower our citizens with the skills they need to work harder, learn more, reach higher. But while the means will change, our purpose endures: a nation that rewards the effort and determination of every single American. That is what this moment requires. That is what will give real meaning to our creed.  We, the people, still believe that every citizen deserves a basic measure of security and dignity. We must make the hard choices to reduce the cost of health care and the size of our deficit. But we reject the belief that America must choose between caring for the generation that built this country and investing in the generation that will build its future. For we remember the lessons of our past, when twilight years were spent in poverty and parents of a child with a disability had nowhere to turn.  We do not believe that in this country freedom is reserved for the lucky, or happiness for the few. We recognize that no matter how responsibly we live our lives, any one of us at any time may face a job loss or a sudden illness or a home swept away in a terrible storm. The commitments we make to each other through Medicare and Medicaid and Social Security, these things do not sap our initiative, they strengthen us. They do not make us a nation of takers; they free us to take the risks that make this country great.  We, the people, still believe that our obligations as Americans are not just to ourselves, but to all posterity. We will respond to the threat of climate change, knowing that the failure to do so would betray our children and future generations. Some may still deny the overwhelming judgment of science, but none can avoid the devastating impact of raging fires and crippling drought and more powerful storms.  The path towards sustainable energy sources will be long and sometimes difficult. But America cannot resist this transition, we must lead it. We cannot cede to other nations the technology that will power new jobs and new industries, we must claim its promise. That's how we will maintain our economic vitality and our national treasure—our forests and waterways, our crop lands and snow-capped peaks. That is how we will preserve our planet, commanded to our care by God. That's what will lend meaning to the creed our fathers once declared.  We, the people, still believe that enduring security and lasting peace do not require perpetual war. Our brave men and women in uniform, tempered by the flames of battle, are unmatched in skill and courage. Our citizens, seared by the memory of those we have lost, know too well the price that is paid for liberty. The knowledge of their sacrifice will keep us forever vigilant against those who would do us harm. But we are also heirs to those who won the peace and not just the war; who turned sworn enemies into the surest of friends—and we must carry those lessons into this time as well.  We will defend our people and uphold our values through strength of arms and rule of law. We will show the courage to try and resolve our differences with other nations peacefully—not because we are naive about the dangers we face, but because engagement can more durably lift suspicion and fear.  America will remain the anchor of strong alliances in every corner of the globe. And we will renew those institutions that extend our capacity to manage crisis abroad, for no one has a greater stake in a peaceful world than its most powerful nation. We will support democracy from Asia to Africa, from the Americas to the Middle East, because our interests and our conscience compel us to act on behalf of those who long for freedom. And we must be a source of hope to the poor, the sick, the marginalized, the victims of prejudice—not out of mere charity, but because peace in our time requires the constant advance of those principles that our common creed describes: tolerance and opportunity, human dignity and justice.  We, the people, declare today that the most evident of truths—that all of us are created equal—is the star that guides us still; just as it guided our forebears through Seneca Falls and Selma and Stonewall; just as it guided all those men and women, sung and unsung, who left footprints along this great Mall, to hear a preacher say that we cannot walk alone; to hear a King proclaim that our individual freedom is inextricably bound to the freedom of every soul on Earth.  It is now our generation's task to carry on what those pioneers began. For our journey is not complete until our wives, our mothers and daughters can earn a living equal to their efforts. Our journey is not complete until our gay brothers and sisters are treated like anyone else under the law—for if we are truly created equal, then surely the love we commit to one another must be equal as well. Our journey is not complete until no citizen is forced to wait for hours to exercise the right to vote. Our journey is not complete until we find a better way to welcome the striving, hopeful immigrants who still see America as a land of opportunity—until bright young students and engineers are enlisted in our workforce rather than expelled from our country. Our journey is not complete until all our children, from the streets of Detroit to the hills of Appalachia, to the quiet lanes of Newtown, know that they are cared for and cherished and always safe from harm.  That is our generation's task—to make these words, these rights, these values of life and liberty and the pursuit of happiness real for every American. Being true to our founding documents does not require us to agree on every contour of life. It does not mean we all define liberty in exactly the same way or follow the same precise path to happiness. Progress does not compel us to settle centuries-long debates about the role of government for all time, but it does require us to act in our time.  For now decisions are upon us and we cannot afford delay. We cannot mistake absolutism for principle or substitute spectacle for politics or treat name-calling as reasoned debate. We must act, knowing that our work will be imperfect. We must act, we must act knowing that today's victories will be only partial and that it will be up to those who stand here in 4 years and 40 years and 400 years hence to advance the timeless spirit once conferred to us in a spare Philadelphia hall.  My fellow Americans, the oath I have sworn before you today, like the one recited by others who serve in this Capitol, was an oath to God and country, not party or faction. And we must faithfully execute that pledge during the duration of our service. But the words I spoke today are not so different from the oath that is taken each time a soldier signs up for duty or an immigrant realizes her dream. My oath is not so different from the pledge we all make to the flag that waves above and that fills our hearts with pride.  They are the words of citizens and they represent our greatest hope. You and I, as citizens, have the power to set this country's course. You and I, as citizens, have the obligation to shape the debates of our time—not only with the votes we cast, but with the voices we lift in defense of our most ancient values and enduring ideals.  Let us, each of us, now embrace with solemn duty and awesome joy what is our lasting birthright. With common effort and common purpose, with passion and dedication, let us answer the call of history and carry into an uncertain future that precious light of freedom.  Thank you. God bless you, and may He forever bless these United States of America.", 'NaN', "Democrat", "Incumbent_Party"]]
trump = ['NaN', 2017, "Donald J. Trump", "Chief Justice Roberts, President Carter, President Clinton, President Bush, President Obama, fellow Americans, and people of the world: Thank you.  We, the citizens of America, are now joined in a great national effort to rebuild our country and restore its promise for all of our people. Together, we will determine the course of America and the world for many, many years to come. We will face challenges, we will confront hardships, but we will get the job done.  Every 4 years, we gather on these steps to carry out the orderly and peaceful transfer of power, and we are grateful to President Obama and First Lady Michelle Obama for their gracious aid throughout this transition. They have been magnificent. Thank you.  Today's ceremony, however, has very special meaning. Because today we are not merely transferring power from one administration to another or from one party to another, but we are transferring power from Washington, DC, and giving it back to you, the people.  For too long, a small group in our Nation's Capital has reaped the rewards of Government while the people have borne the cost. Washington flourished, but the people did not share in its wealth. Politicians prospered, but the jobs left, and the factories closed. The establishment protected itself, but not the citizens of our country. Their victories have not been your victories; their triumphs have not been your triumphs; and while they celebrated in our Nation's Capital, there was little to celebrate for struggling families all across our land.  That all changes, starting right here and right now, because this moment is your moment: It belongs to you. It belongs to everyone gathered here today and everyone watching all across America. This is your day. This is your celebration. And this, the United States of America, is your country.  What truly matters is not which party controls our Government, but whether our Government is controlled by the people. January 20, 2017, will be remembered as the day the people became the rulers of this Nation again. The forgotten men and women of our country will be forgotten no longer. Everyone is listening to you now.  You came by the tens of millions to become part of a historic movement the likes of which the world has never seen before. At the center of this movement is a crucial conviction: that a nation exists to serve its citizens. Americans want great schools for their children, safe neighborhoods for their families, and good jobs for themselves. These are just and reasonable demands of righteous people and a righteous public.  But for too many of our citizens, a different reality exists: Mothers and children trapped in poverty in our inner cities; rusted-out factories scattered like tombstones across the landscape of our Nation; an education system, flush with cash, but which leaves our young and beautiful students deprived of all knowledge; and the crime and the gangs and the drugs that have stolen too many lives and robbed our country of so much unrealized potential.  This American carnage stops right here and stops right now. We are one Nation, and their pain is our pain, their dreams are our dreams, and their success will be our success. We share one heart, one home, and one glorious destiny.  The oath of office I take today is an oath of allegiance to all Americans.  For many decades, we've enriched foreign industry at the expense of American industry, subsidized the armies of other countries while allowing for the very sad depletion of our military. We've defended other nations' borders while refusing to defend our own and spent trillions and trillions of dollars overseas while America's infrastructure has fallen into disrepair and decay. We've made other countries rich while the wealth, strength, and confidence of our country has dissipated over the horizon.  One by one, the factories shuttered and left our shores, with not even a thought about the millions and millions of American workers that were left behind. The wealth of our middle class has been ripped from their homes and then redistributed all across the world.  But that is the past. And now we are looking only to the future.  We, assembled here today, are issuing a new decree to be heard in every city, in every foreign capital, and in every hall of power. From this day forward, a new vision will govern our land. From this this day forward, it's going to be only America first. America first.  Every decision on trade, on taxes, on immigration, on foreign affairs, will be made to benefit American workers and American families.  We must protect our borders from the ravages of other countries making our products, stealing our companies, and destroying our jobs. Protection will lead to great prosperity and strength. I will fight for you with every breath in my body, and I will never, ever let you down.  America will start winning again, winning like never before. We will bring back our jobs. We will bring back our borders. We will bring back our wealth. And we will bring back our dreams.  We will build new roads and highways and bridges and airports and tunnels and railways all across our wonderful Nation.  We will get our people off of welfare and back to work, rebuilding our country with American hands and American labor. We will follow two simple rules: Buy American and hire American.  We will seek friendship and good will with the nations of the world, but we do so with the understanding that it is the right of all nations to put their own interests first. We do not seek to impose our way of life on anyone, but rather to let it shine as an example—we will shine—for everyone to follow.  We will reinforce old alliances and form new ones and unite the civilized world against radical Islamic terrorism, which we will eradicate completely from the face of the Earth.  At the bedrock of our politics will be a total allegiance to the United States of America, and through our loyalty to our country, we will rediscover our loyalty to each other. When you open your heart to patriotism, there is no room for prejudice. The Bible tells us, How good and pleasant it is when God's people live together in unity. We must speak our minds openly, debate our disagreements honestly, but always pursue solidarity. When America is united, America is totally unstoppable. There should be no fear: We are protected, and we will always be protected. We will be protected by the great men and women of our military and law enforcement, and most importantly, we will be protected by God.  Finally, we must think big and dream even bigger. In America, we understand that a nation is only living as long as it is striving.  We will no longer accept politicians who are all talk and no action, constantly complaining, but never doing anything about it. The time for empty talk is over. Now arrives the hour of action.  Do not allow anyone to tell you that it cannot be done. No challenge can match the heart and fight and spirit of America. We will not fail. Our country will thrive and prosper again.  We stand at the birth of a new millennium, ready to unlock the mysteries of space, to free the Earth from the miseries of disease, and to harness the energies, industries, and technologies of tomorrow. A new national pride will stir our souls, lift our sights, and heal our divisions.  It's time to remember that old wisdom our soldiers will never forget: that whether we are Black or Brown or White, we all bleed the same red blood of patriots, we all enjoy the same glorious freedoms, and we all salute the same great American flag.  And whether a child is born in the urban sprawl of Detroit or the windswept plains of Nebraska, they look up at the same night sky, they fill their heart with the same dreams, and they are infused with the breath of life by the same almighty Creator.  So to all Americans in every city near and far, small and large, from mountain to mountain, from ocean to ocean, hear these words: You will never be ignored again. Your voice, your hopes, and your dreams will define our American destiny. And your courage and goodness and love will forever guide us along the way.  Together, we will make America strong again. We will make America wealthy again. We will make America proud again. We will make America safe again.  And, yes, together, we will make America great again. Thank you. God bless you, and God bless America. Thank you. God bless America.", 'NaN', 'Republican', 'New_Party,']
barack3.append(trump)
ob_tr = pd.DataFrame(barack3, columns = ['urls', 'years', 'pres', 'texts', 'wordcounts', 'party', 'change'], index = [56, 57])
ob_tr

urls  years             pres  \
56  NaN   2013     Barack Obama   
57  NaN   2017  Donald J. Trump   

                                                texts wordcounts       party  \
56  Thank you. Thank you so much.  Vice President ...        NaN    Democrat   
57  Chief Justice Roberts, President Carter, Presi...        NaN  Republican   

             change  
56  Incumbent_Party  
57       New_Party,

In [21]:
frames = [updated_df, ob_tr]
updated_df = pd.concat(frames)
updated_df.reset_index(drop=True)
updated_df.tail()


urls  years             pres  \
53  http://avalon.law.yale.edu/21st_century/gbush1...   2001   George W. Bush   
54  http://avalon.law.yale.edu/21st_century/gbush2...   2005   George W. Bush   
55  http://avalon.law.yale.edu/21st_century/obama.asp   2009     Barack Obama   
56                                                NaN   2013     Barack Obama   
57                                                NaN   2017  Donald J. Trump   

                                                texts wordcounts       party  \
53  The Avalon Project at Yale Law School    Inaug...       1595  Republican   
54  The Avalon Project at Yale Law School    Secon...       2073  Republican   
55  The Avalon Project at Yale Law School    Inaug...       2417    Democrat   
56  Thank you. Thank you so much.  Vice President ...        NaN    Democrat   
57  Chief Justice Roberts, President Carter, Presi...        NaN  Republican   

             change  
53        New_Party  
54  Incumbent_Party  
55        New_Party  
56  Incumbent_Party  
57       New_Party,

## Recount the words
Some of the texts were not there in the previous df, so now to update the wordcounts:

In [22]:
# A function to count the words
def wordcounter(text):
    words = text.split()
    count = 0
    for word in words:
        count += 1
    return count

new_counts = []
for ind in range(len(updated_df)):
    count = wordcounter(updated_df['texts'][ind])
    # this attempt below (commented out) returned a warning about chained indexing
    # I did not have time to read and understand it, so I simply made a new colum with updated
    # wordcounts
    # updated_df['wordcounts'][ind] = count
    new_counts.append(count)
#     print(count)
    
updated_df['new counts'] = new_counts

updated_df.tail()

urls  years             pres  \
53  http://avalon.law.yale.edu/21st_century/gbush1...   2001   George W. Bush   
54  http://avalon.law.yale.edu/21st_century/gbush2...   2005   George W. Bush   
55  http://avalon.law.yale.edu/21st_century/obama.asp   2009     Barack Obama   
56                                                NaN   2013     Barack Obama   
57                                                NaN   2017  Donald J. Trump   

                                                texts wordcounts       party  \
53  The Avalon Project at Yale Law School    Inaug...       1595  Republican   
54  The Avalon Project at Yale Law School    Secon...       2073  Republican   
55  The Avalon Project at Yale Law School    Inaug...       2417    Democrat   
56  Thank you. Thank you so much.  Vice President ...        NaN    Democrat   
57  Chief Justice Roberts, President Carter, Presi...        NaN  Republican   

             change  new counts  
53        New_Party        1595  
54  Incumbent_Party        2073  
55        New_Party        2417  
56  Incumbent_Party        2090  
57       New_Party,        1456

## A bit more munging
I have only a few more things I want to do before analysis:

1. I want to pull out some of the text from the current "texts" column, specifically words like "Avalon," "Yale," and "Inaugural Address," and other metadata snippets that I failed to remove with my first cleaning attempts
2. I want to organize the df by date, because currently it's not quite chronological and that's bugging me. 
3. I want to start analysis!

## SO Frustrating
I have spent 2 hours trying to figure out how to remove substrings from strings within the panda series in updated_df['texts']. The str.replace() function seems the most promising, but it won't work with any substring containing spaces. So now I'm stuck; I know that removing substrings from strings is often a matter of slicing out the substring, but I don't know how to do that within pandas dataframe, and honestly this str.replace() function makes me think that it's unnecessary. I don't know. Yet another couple hours wasted trying to figure out a munging problems!! GRRR!


In [23]:
# remove_strings = ['The Avalon Project at Yale Law School', 'Inaugural Address of']
# for string in updated_df['texts']:
#     print(type(string))
#     for sub_str in remove_strings:
#         print(sub_str)
#         if sub_str in string:
#             print(string[:50])
#             print("it's here")
#             string.replace(sub_str, '')
#         else:
#             continue

remove_strings = ['The Avalon Project', 'at Yale Law School', 'First Inaugural Address', 'Second Inaugural Address', 'Inaugural Address']
for string in remove_strings:
    print(string)
    updated_df['texts'] = updated_df['texts'].str.replace(string, ' ')
            
for string in pres_list:
    updated_df['texts'] = updated_df['texts'].str.replace(string,  ' ')
        

# remove_substr('Washington')
# updated_df['texts'] = updated_df['texts'].str.replace('Washington', '')
# updated_df

The Avalon Project
at Yale Law School
First Inaugural Address
Second Inaugural Address
Inaugural Address


In [24]:
updated_df


urls  years  \
0   http://avalon.law.yale.edu/18th_century/wash1.asp   1789   
1   http://avalon.law.yale.edu/18th_century/wash2.asp   1793   
2   http://avalon.law.yale.edu/18th_century/adams.asp   1797   
3   http://avalon.law.yale.edu/19th_century/jefina...   1801   
4   http://avalon.law.yale.edu/19th_century/jefina...   1805   
5   http://avalon.law.yale.edu/19th_century/madiso...   1809   
6   http://avalon.law.yale.edu/19th_century/madiso...   1813   
7   http://avalon.law.yale.edu/19th_century/monroe...   1817   
8   http://avalon.law.yale.edu/19th_century/monroe...   1821   
9   http://avalon.law.yale.edu/19th_century/qadams...   1825   
10  http://avalon.law.yale.edu/19th_century/jackso...   1829   
11  http://avalon.law.yale.edu/19th_century/jackso...   1833   
12  http://avalon.law.yale.edu/19th_century/vanbur...   1837   
13  http://avalon.law.yale.edu/19th_century/harris...   1841   
14   http://avalon.law.yale.edu/19th_century/polk.asp   1845   
15  http://avalon.law.yale.edu/19th_century/taylor...   1849   
16  http://avalon.law.yale.edu/19th_century/pierce...   1853   
17  http://avalon.law.yale.edu/19th_century/buchan...   1857   
18  http://avalon.law.yale.edu/19th_century/lincol...   1861   
19  http://avalon.law.yale.edu/19th_century/lincol...   1865   
20  http://avalon.law.yale.edu/19th_century/grant1...   1869   
21  http://avalon.law.yale.edu/19th_century/grant2...   1873   
22  http://avalon.law.yale.edu/19th_century/hayes.asp   1877   
23  http://avalon.law.yale.edu/19th_century/garfie...   1881   
24  http://avalon.law.yale.edu/19th_century/cleve1...   1885   
25  http://avalon.law.yale.edu/19th_century/cleve2...   1893   
26  http://avalon.law.yale.edu/19th_century/harris...   1889   
27  http://avalon.law.yale.edu/19th_century/mckin1...   1897   
28  http://avalon.law.yale.edu/19th_century/mckin2...   1901   
29  http://avalon.law.yale.edu/20th_century/troos.asp   1905   
30   http://avalon.law.yale.edu/20th_century/taft.asp   1909   
31  http://avalon.law.yale.edu/20th_century/wilson...   1913   
32  http://avalon.law.yale.edu/20th_century/wilson...   1917   
33  http://avalon.law.yale.edu/20th_century/hardin...   1921   
34  http://avalon.law.yale.edu/20th_century/coolid...   1925   
35  http://avalon.law.yale.edu/20th_century/hoover...   1929   
36  http://avalon.law.yale.edu/20th_century/froos1...   1933   
37  http://avalon.law.yale.edu/20th_century/froos2...   1937   
38  http://avalon.law.yale.edu/20th_century/froos3...   1941   
39  http://avalon.law.yale.edu/20th_century/froos4...   1945   
40  http://avalon.law.yale.edu/20th_century/truman...   1949   
41  http://avalon.law.yale.edu/20th_century/eisen1...   1953   
42  http://avalon.law.yale.edu/20th_century/eisen2...   1957   
43  http://avalon.law.yale.edu/20th_century/kenned...   1961   
44  http://avalon.law.yale.edu/20th_century/johnso...   1965   
45  http://avalon.law.yale.edu/20th_century/nixon1...   1969   
46  http://avalon.law.yale.edu/20th_century/nixon2...   1973   
47  http://avalon.law.yale.edu/20th_century/carter...   1977   
48  http://avalon.law.yale.edu/20th_century/reagan...   1981   
49  http://avalon.law.yale.edu/20th_century/reagan...   1985   
50   http://avalon.law.yale.edu/20th_century/bush.asp   1989   
51  http://avalon.law.yale.edu/20th_century/clinto...   1993   
52  http://avalon.law.yale.edu/20th_century/clinto...   1997   
53  http://avalon.law.yale.edu/21st_century/gbush1...   2001   
54  http://avalon.law.yale.edu/21st_century/gbush2...   2005   
55  http://avalon.law.yale.edu/21st_century/obama.asp   2009   
56                                                NaN   2013   
57                                                NaN   2017   

                      pres                                              texts  \
0        George Washington    of    THE CITY OF NEW YORK THURSDAY, APRIL 3...   
1        George Washington    of    THE CITY OF PHILADELPHIA MONDAY, MARCH...   
2               John Adams  ©

In [25]:
updated_df.sort_values(by='years', ascending= True)

urls  years  \
0   http://avalon.law.yale.edu/18th_century/wash1.asp   1789   
1   http://avalon.law.yale.edu/18th_century/wash2.asp   1793   
2   http://avalon.law.yale.edu/18th_century/adams.asp   1797   
3   http://avalon.law.yale.edu/19th_century/jefina...   1801   
4   http://avalon.law.yale.edu/19th_century/jefina...   1805   
5   http://avalon.law.yale.edu/19th_century/madiso...   1809   
6   http://avalon.law.yale.edu/19th_century/madiso...   1813   
7   http://avalon.law.yale.edu/19th_century/monroe...   1817   
8   http://avalon.law.yale.edu/19th_century/monroe...   1821   
9   http://avalon.law.yale.edu/19th_century/qadams...   1825   
10  http://avalon.law.yale.edu/19th_century/jackso...   1829   
11  http://avalon.law.yale.edu/19th_century/jackso...   1833   
12  http://avalon.law.yale.edu/19th_century/vanbur...   1837   
13  http://avalon.law.yale.edu/19th_century/harris...   1841   
14   http://avalon.law.yale.edu/19th_century/polk.asp   1845   
15  http://avalon.law.yale.edu/19th_century/taylor...   1849   
16  http://avalon.law.yale.edu/19th_century/pierce...   1853   
17  http://avalon.law.yale.edu/19th_century/buchan...   1857   
18  http://avalon.law.yale.edu/19th_century/lincol...   1861   
19  http://avalon.law.yale.edu/19th_century/lincol...   1865   
20  http://avalon.law.yale.edu/19th_century/grant1...   1869   
21  http://avalon.law.yale.edu/19th_century/grant2...   1873   
22  http://avalon.law.yale.edu/19th_century/hayes.asp   1877   
23  http://avalon.law.yale.edu/19th_century/garfie...   1881   
24  http://avalon.law.yale.edu/19th_century/cleve1...   1885   
26  http://avalon.law.yale.edu/19th_century/harris...   1889   
25  http://avalon.law.yale.edu/19th_century/cleve2...   1893   
27  http://avalon.law.yale.edu/19th_century/mckin1...   1897   
28  http://avalon.law.yale.edu/19th_century/mckin2...   1901   
29  http://avalon.law.yale.edu/20th_century/troos.asp   1905   
30   http://avalon.law.yale.edu/20th_century/taft.asp   1909   
31  http://avalon.law.yale.edu/20th_century/wilson...   1913   
32  http://avalon.law.yale.edu/20th_century/wilson...   1917   
33  http://avalon.law.yale.edu/20th_century/hardin...   1921   
34  http://avalon.law.yale.edu/20th_century/coolid...   1925   
35  http://avalon.law.yale.edu/20th_century/hoover...   1929   
36  http://avalon.law.yale.edu/20th_century/froos1...   1933   
37  http://avalon.law.yale.edu/20th_century/froos2...   1937   
38  http://avalon.law.yale.edu/20th_century/froos3...   1941   
39  http://avalon.law.yale.edu/20th_century/froos4...   1945   
40  http://avalon.law.yale.edu/20th_century/truman...   1949   
41  http://avalon.law.yale.edu/20th_century/eisen1...   1953   
42  http://avalon.law.yale.edu/20th_century/eisen2...   1957   
43  http://avalon.law.yale.edu/20th_century/kenned...   1961   
44  http://avalon.law.yale.edu/20th_century/johnso...   1965   
45  http://avalon.law.yale.edu/20th_century/nixon1...   1969   
46  http://avalon.law.yale.edu/20th_century/nixon2...   1973   
47  http://avalon.law.yale.edu/20th_century/carter...   1977   
48  http://avalon.law.yale.edu/20th_century/reagan...   1981   
49  http://avalon.law.yale.edu/20th_century/reagan...   1985   
50   http://avalon.law.yale.edu/20th_century/bush.asp   1989   
51  http://avalon.law.yale.edu/20th_century/clinto...   1993   
52  http://avalon.law.yale.edu/20th_century/clinto...   1997   
53  http://avalon.law.yale.edu/21st_century/gbush1...   2001   
54  http://avalon.law.yale.edu/21st_century/gbush2...   2005   
55  http://avalon.law.yale.edu/21st_century/obama.asp   2009   
56                                                NaN   2013   
57                                                NaN   2017   

                      pres                                              texts  \
0        George Washington    of    THE CITY OF NEW YORK THURSDAY, APRIL 3...   
1        George Washington    of    THE CITY OF PHILADELPHIA MONDAY, MARCH...   
2               John Adams  ©

In [26]:
updated_df.reset_index(drop=True)

urls  years  \
0   http://avalon.law.yale.edu/18th_century/wash1.asp   1789   
1   http://avalon.law.yale.edu/18th_century/wash2.asp   1793   
2   http://avalon.law.yale.edu/18th_century/adams.asp   1797   
3   http://avalon.law.yale.edu/19th_century/jefina...   1801   
4   http://avalon.law.yale.edu/19th_century/jefina...   1805   
5   http://avalon.law.yale.edu/19th_century/madiso...   1809   
6   http://avalon.law.yale.edu/19th_century/madiso...   1813   
7   http://avalon.law.yale.edu/19th_century/monroe...   1817   
8   http://avalon.law.yale.edu/19th_century/monroe...   1821   
9   http://avalon.law.yale.edu/19th_century/qadams...   1825   
10  http://avalon.law.yale.edu/19th_century/jackso...   1829   
11  http://avalon.law.yale.edu/19th_century/jackso...   1833   
12  http://avalon.law.yale.edu/19th_century/vanbur...   1837   
13  http://avalon.law.yale.edu/19th_century/harris...   1841   
14   http://avalon.law.yale.edu/19th_century/polk.asp   1845   
15  http://avalon.law.yale.edu/19th_century/taylor...   1849   
16  http://avalon.law.yale.edu/19th_century/pierce...   1853   
17  http://avalon.law.yale.edu/19th_century/buchan...   1857   
18  http://avalon.law.yale.edu/19th_century/lincol...   1861   
19  http://avalon.law.yale.edu/19th_century/lincol...   1865   
20  http://avalon.law.yale.edu/19th_century/grant1...   1869   
21  http://avalon.law.yale.edu/19th_century/grant2...   1873   
22  http://avalon.law.yale.edu/19th_century/hayes.asp   1877   
23  http://avalon.law.yale.edu/19th_century/garfie...   1881   
24  http://avalon.law.yale.edu/19th_century/cleve1...   1885   
25  http://avalon.law.yale.edu/19th_century/cleve2...   1893   
26  http://avalon.law.yale.edu/19th_century/harris...   1889   
27  http://avalon.law.yale.edu/19th_century/mckin1...   1897   
28  http://avalon.law.yale.edu/19th_century/mckin2...   1901   
29  http://avalon.law.yale.edu/20th_century/troos.asp   1905   
30   http://avalon.law.yale.edu/20th_century/taft.asp   1909   
31  http://avalon.law.yale.edu/20th_century/wilson...   1913   
32  http://avalon.law.yale.edu/20th_century/wilson...   1917   
33  http://avalon.law.yale.edu/20th_century/hardin...   1921   
34  http://avalon.law.yale.edu/20th_century/coolid...   1925   
35  http://avalon.law.yale.edu/20th_century/hoover...   1929   
36  http://avalon.law.yale.edu/20th_century/froos1...   1933   
37  http://avalon.law.yale.edu/20th_century/froos2...   1937   
38  http://avalon.law.yale.edu/20th_century/froos3...   1941   
39  http://avalon.law.yale.edu/20th_century/froos4...   1945   
40  http://avalon.law.yale.edu/20th_century/truman...   1949   
41  http://avalon.law.yale.edu/20th_century/eisen1...   1953   
42  http://avalon.law.yale.edu/20th_century/eisen2...   1957   
43  http://avalon.law.yale.edu/20th_century/kenned...   1961   
44  http://avalon.law.yale.edu/20th_century/johnso...   1965   
45  http://avalon.law.yale.edu/20th_century/nixon1...   1969   
46  http://avalon.law.yale.edu/20th_century/nixon2...   1973   
47  http://avalon.law.yale.edu/20th_century/carter...   1977   
48  http://avalon.law.yale.edu/20th_century/reagan...   1981   
49  http://avalon.law.yale.edu/20th_century/reagan...   1985   
50   http://avalon.law.yale.edu/20th_century/bush.asp   1989   
51  http://avalon.law.yale.edu/20th_century/clinto...   1993   
52  http://avalon.law.yale.edu/20th_century/clinto...   1997   
53  http://avalon.law.yale.edu/21st_century/gbush1...   2001   
54  http://avalon.law.yale.edu/21st_century/gbush2...   2005   
55  http://avalon.law.yale.edu/21st_century/obama.asp   2009   
56                                                NaN   2013   
57                                                NaN   2017   

                      pres                                              texts  \
0        George Washington    of    THE CITY OF NEW YORK THURSDAY, APRIL 3...   
1        George Washington    of    THE CITY OF PHILADELPHIA MONDAY, MARCH...   
2               John Adams  ©

## Munging is pretty much complete
Now I need to start actually doing the analysis. We start with some of the basic wordcounting/ tokenizing/ preprocessing that will go into the work. I will do that in a different notebook, as this one is getting long. 

In [27]:
# Save the dataframe to csv
updated_df.to_csv('data/inaugurals_complete.csv', index = False)